In [3]:
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target,
                                                    train_size=0.75, test_size=0.25)

pipeline_optimizer = TPOTClassifier(generations=5, population_size=20, cv=5,
                                    random_state=42, verbosity=2)

pipeline_optimizer.fit(X_train, y_train)
print(pipeline_optimizer.score(X_test, y_test))
pipeline_optimizer.export('tpot_exported_pipeline.py')

Generation 1 - Current best internal CV score: 0.9799476800220294
Generation 2 - Current best internal CV score: 0.9799476800220294
Generation 3 - Current best internal CV score: 0.9799476800220294
Generation 4 - Current best internal CV score: 0.9799476800220294
Generation 5 - Current best internal CV score: 0.9799559410711828

Best pipeline: ExtraTreesClassifier(MaxAbsScaler(input_matrix), bootstrap=False, criterion=gini, max_features=0.3, min_samples_leaf=3, min_samples_split=4, n_estimators=100)
0.9755555555555555
